1.Use the beautifulsoup library to scrape the data (from menuism.com like the link above) on state names and corresponding number of store locations, for the following chains:
Starbucks

Dunkin’ Donuts

Peet’s Coffee & Tea

Tim Horton’s

Panera Bread

Caribou Coffee

Au Bon Pain

The Coffee Bean & Tea Leaf

McDonald’s

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
########################################## Scrape Location Counts ###################################################

response = requests.get('https://www.menuism.com/restaurant-locations/starbucks-coffee-39564')

# Check if the request was successful
if response.status_code == 200:
    page_content = response.content
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


Failed to retrieve the webpage. Status code: 502


In [3]:
soup = BeautifulSoup(page_content, 'html.parser')

store_name = soup.find('h1').find('span', {'itemprop': 'name'}).text
a_titles = soup.find_all('a', title = lambda title: title and title.endswith(', us'))
stores = {}
states_map = {}
for title in a_titles:
    code = title.get('href')[-2:].upper()
    splits = title.text.split(' ')
    ignore_list = store_name.split(' ')
    state = ' '.join(filter(lambda x : x not in ignore_list and x!='locations' and x[0]!='(', splits))
    locations = splits[-1][1:-1]
    stores[state] = locations
    states_map[state] = code

NameError: name 'page_content' is not defined

A function stateabb() that takes a state name (assume it’s spelled correctly) and converts it to its state abbreviation.

In [4]:
def stateabb(state):
  if state == 'Mississippi':
    return 'MS'
  return states_map[state]

Parsing, merging and tiding data so that I have a row for each state and two columns: state abbrevation, location count.

In [5]:
stores_df = pd.DataFrame(list(stores.items()), columns=['State', 'Locations']) # stores location numbers with the state abbreviations
stores_df['State'] = stores_df['State'].apply(stateabb)
print("Store Name = "+store_name)
print(stores_df)

NameError: name 'stores' is not defined

Scraping the state names and populations from wikipedia page. Converting the state names to abbreviations and merging the data with the coffee dataset.

In [6]:
# Supplemental Data

response = requests.get('https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population')

# Checking if the request was successful
if response.status_code == 200:
    page_content = response.content
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

soup = BeautifulSoup(page_content, 'html.parser')
table = soup.find('table')
state_population = {}

# Iterating through the rows to extract data from the second column
for row in table.find_all('tr'):
    columns = row.find_all('td')
    if len(columns) > 0:  # Ensure there's at least a second column
        state_population[columns[2].text.replace('\xa0', '').replace('\n', '')] = columns[3].text.replace('\n', '')

del state_population['Puerto Rico'] #filtering the non states
del state_population['Guam']
del state_population['U.S. Virgin Islands']
del state_population['American Samoa']
del state_population['Northern Mariana Islands']
del state_population['Contiguous United States']
del state_population['The fifty states']
del state_population['Fifty states + D.C.']
del state_population['Total U.S. (including D.C. and territories)']

Creating a region variable in my dataset according to the scheme on the wikipedia page: Northeast, Midwest, South, West.

In [7]:
def region(state):
  ne = ["Connecticut", "Maine", "Massachusetts", "New Hampshire", "Vermont", "Rhode Island", "New Jersey", "New York", "Pennsylvania"]
  mw = ["Illinois", "Indiana", "Michigan", "Ohio", "Wisconsin", "Iowa", "Kansas", "Minnesota", "Missouri", "Nebraska", "North Dakota", "South Dakota"]
  w = ["Arizona", "Colorado", "Idaho", "Montana", "Nevada", "New Mexico", "Utah", "Wyoming", "Alaska", "California", "Hawaii", "Oregon", "Washington"]
  if state in ne:
    return "Northeast"
  if state in mw:
    return "Midwest"
  if state in w:
    return "West"
  return "South"



In [8]:
# Print the data from the second column
populations_df = pd.DataFrame(list(state_population.items()), columns=['State', 'Population'])
populations_df['region'] = populations_df['State'].apply(region)
populations_df['State'] = populations_df['State'].str.strip()
populations_df['State'] = populations_df['State'].map(states_map)
result_df = pd.merge(stores_df, populations_df, on='State', how='inner')
print(result_df)

NameError: name 'states_map' is not defined

Automating the functions to automate all functions so that it take just a URL to data scrape and give a cleaned out data.

In [16]:
# Automate
store_urls = ('https://www.menuism.com/restaurant-locations/starbucks-coffee-39564',
              'https://www.menuism.com/restaurant-locations/dunkin-donuts-181624',
              'https://www.menuism.com/restaurant-locations/peets-coffee-tea-84051',
              'https://www.menuism.com/restaurant-locations/tim-hortons-190025',
              'https://www.menuism.com/restaurant-locations/panera-bread-4258',
              'https://www.menuism.com/restaurant-locations/caribou-coffee-164861',
              'https://www.menuism.com/restaurant-locations/au-bon-pain-69342',
              'https://www.menuism.com/restaurant-locations/the-coffee-bean-tea-leaf-165988',
              'https://www.menuism.com/restaurant-locations/mcdonalds-21019')

def scrape_website(store_url):
  response = requests.get(store_url)

  if response.status_code == 200:
      page_content = response.content
  else:
      print(f"Error. Status code: {response.status_code}")

  soup = BeautifulSoup(page_content, 'html.parser')

  store_name = soup.find('h1').find('span', {'itemprop': 'name'}).text
  a_titles = soup.find_all('a', title = lambda title: title and title.endswith(', us'))
  stores = {}
  states_map = {}
  for title in a_titles:
      code = title.get('href')[-2:].upper()
      splits = title.text.split(' ')
      ignore_list = store_name.split(' ')
      state = ' '.join(filter(lambda x : x not in ignore_list and x!='locations' and x[0]!='(', splits))
      locations = splits[-1][1:-1]
      stores[state] = locations
      states_map[state] = code

  def stateabb(state):
    return states_map[state]

  stores_df = pd.DataFrame(list(stores.items()), columns=['State', 'Locations'])
  stores_df['State'] = stores_df['State'].apply(stateabb)
  stores_df['Company'] = store_name
  print("Store Name = "+store_name)
  print(stores_df)

In [10]:
scrape_website('https://www.menuism.com/restaurant-locations/dunkin-donuts-181624') #testing the function with one link

Error. Status code: 502


UnboundLocalError: cannot access local variable 'page_content' where it is not associated with a value

In [17]:
url = "Starbucks Coffee Locations Near Me + Reviews & Menu.html" #Working with starbucks data
page = open(url)
soup1 = BeautifulSoup(page.read())

In [18]:
def scrape_website(soup): #Modifying function to take in html as dataset to webscrape
  # response = requests.get(store_url)

  # if response.status_code == 200:
  #     page_content = response.content
  # else:
  #     print(f"Error. Status code: {response.status_code}")

  # soup = BeautifulSoup(soup1, 'html.parser')

  store_name = soup.find('h1').find('span', {'itemprop': 'name'}).text
  a_titles = soup.find_all('a', title = lambda title: title and title.endswith(', us'))
  stores = {}
  states_map = {}
  for title in a_titles:
      code = title.get('href')[-2:].upper()
      splits = title.text.split(' ')
      ignore_list = store_name.split(' ')
      state = ' '.join(filter(lambda x : x not in ignore_list and x!='locations' and x[0]!='(', splits))
      locations = splits[-1][1:-1]
      stores[state] = locations
      states_map[state] = code

  def stateabb(state):
    return states_map[state]

  stores_df = pd.DataFrame(list(stores.items()), columns=['State', 'Locations'])
  stores_df['State'] = stores_df['State'].apply(stateabb)
  stores_df['Company'] = store_name
  print("Store Name = "+store_name)
  print(stores_df)

In [11]:
scrape_website(soup1)

NameError: name 'soup1' is not defined

5.Find the revenue, stock price, or your financial metric of choice for each of the companies listed above (if you can find a website to scrape these from that’s great!…but it’s okay if you manually enter these into R). Merge these values into your big dataset. Note: these values may be repeated for each state.

In [12]:
url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=SBUX&interval=5min&apikey=XFYGY6J374NSETZP'
r = requests.get(url)
data = r.json()

print(data) #requesting yearly revenue data for starbucks with a free API key


{'symbol': 'SBUX', 'annualReports': [{'fiscalDateEnding': '2022-09-30', 'reportedCurrency': 'USD', 'grossProfit': '8371100000', 'totalRevenue': '32250300000', 'costOfRevenue': '23879200000', 'costofGoodsAndServicesSold': '23879200000', 'operatingIncome': '4617800000', 'sellingGeneralAndAdministrative': '2032000000', 'researchAndDevelopment': 'None', 'operatingExpenses': '3941400000', 'investmentIncomeNet': 'None', 'netInterestIncome': '-385900000', 'interestIncome': '97000000', 'interestExpense': '482900000', 'nonInterestIncome': 'None', 'otherNonOperatingIncome': '97000000', 'depreciation': 'None', 'depreciationAndAmortization': '1447900000', 'incomeBeforeTax': '4230100000', 'incomeTaxExpense': '948500000', 'interestAndDebtExpense': 'None', 'netIncomeFromContinuingOperations': '3283400000', 'comprehensiveIncomeNetOfTax': '2671200000', 'ebit': '4713000000', 'ebitda': '6160900000', 'netIncome': '3281600000'}, {'fiscalDateEnding': '2021-09-30', 'reportedCurrency': 'USD', 'grossProfit': '

Total revenue of Starbucks of year 2022

In [13]:
revenue = data["annualReports"][0]['totalRevenue']
revenue

'32250300000'

To calculate the revenue per state we divide the overall revenue throughtout America by the total number of locations of Starbucks and estimate the revenue per state.(assuming equal revenue per location for analyzing purposes)

Assess and comment on the prevalence of each chain. Some questions to consider (you don’t need to answer all of these and you may come up with your own):
Are some of these chains more prevalent in certain states than others? Possibly despite having less stores overall? Same questions for regions instead of states.<br>
As I was estimating the revenue per location to be equal which resulted in the revenue generated by each state became dependent on the total number of locations each state had. But the number of locations might have be built based on the number of orders and customer traffic. In this case it can be assumed that more number of locations generated more revenue

How does your chosen financial metric change by state and region for each chain? For example, having 5 stores in California is very different from having 5 stores in Wyoming.

Does the distribution of each chain’s stores match population distribution, by both state/region?
Yes, it might be possible to have different no. of locations for each chain

Do the financial data match what you’d expect based on the number and locations of the stores? Why or why not?